In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
import wandb
from wandb.keras import WandbCallback
import os

In [8]:
if os.path.exists('best_model.h5'):
    os.remove('best_model.h5')

In [9]:
seed = 100
tf.random.set_seed(seed)
np.random.seed(seed)

In [28]:
wandb.login()

### Downloading and unzipping iNaturalist Dataset

In [10]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip

In [11]:
!unzip "./nature_12K.zip"

### Preparing data for training

In [12]:
def train_dataset(augmentation=False, batch_size=64):
    dir_train = './inaturalist_12K/train'
    dir_test = './inaturalist_12K/val'

    if augmentation:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                          zoom_range=0.3,
                                          rotation_range=50,
                                          brightness_range=(0.2, 0.8),
                                          shear_range=0.2,
                                          width_shift_range=0.1,
                                          height_shift_range=0.2,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          validation_split=0.1,)
        test_datagen = ImageDataGenerator(rescale=1./255)

    else:
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    train = train_datagen.flow_from_directory(dir_train, target_size=(200, 200), batch_size=batch_size, subset="training")
    val = train_datagen.flow_from_directory(dir_train, target_size=(200, 200), batch_size=batch_size, subset="validation")
    test = test_datagen.flow_from_directory(dir_test, target_size=(200, 200), batch_size=batch_size)
    
    return train, val, test;

In [13]:
def CNN(n_filters, filter_multiplier, dropout, batch_norm, dense_size, act_func= "relu", n_classes=10, image_size=200):
    
    filter_dim = [3, 3, 5, 5, 7]

    model = Sequential()
    for i in range(5):
        #filter_dim = 6-i 
        filter_size = (filter_dim[i], filter_dim[i])
        if i==0:
            model.add(Conv2D(n_filters, filter_size, input_shape=(image_size, image_size, 3), data_format="channels_last"))
        else:
            model.add(Conv2D(n_filters, filter_size))
        if batch_norm:
            model.add(BatchNormalization())
        if act_func == "relu":
            model.add(Activation(act_func))
        if act_func == "leaky":
            model.add(keras.layers.LeakyReLU(alpha=0.3))
            
        model.add(MaxPooling2D(pool_size=(2,2)))
        num_filters = int(n_filters * filter_multiplier)
    
    model.add(Flatten())
    model.add(Dense(dense_size))
    model.add(Dropout(dropout))
    if act_func == "relu":
        model.add(Activation("relu"))
    if act_func == "leaky":
        model.add(keras.layers.LeakyReLU(alpha=0.3))
    model.add(Dense(n_classes))
    model.add(Activation("softmax"))

    return model

### Visualizing some training images

In [19]:
train, val, test = train_dataset(augmentation=False, batch_size=64)
img = train.next()

In [12]:
idx_to_class = {0: 'Amphibia', 1: 'Animalia', 2: 'Arachnida', 3: 'Aves', 4: 'Fungi', 
                  5: 'Insecta', 6: 'Mammalia', 7: 'Mollusca', 8: 'Plantae', 9: 'Reptilia'}

plt.figure(figsize=(15,15))
for i in range(64):
    plt.subplot(8,8,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(img[0][i])
    plt.xlabel(idx_to_class[np.argmax(img[1][i])])
plt.show()

### Function to set WandB run name

In [ ]:
def setWandbName(n_filters, filter_multiplier, augment, dropout, batch_norm):
    
    batch_norm_dict = {True: "Y", False: "N"}
    augment_dict = {True: "Y", False: "N"}

    name = "_".join(["num", str(n_filters), "org", str(filter_multiplier), "aug", augment_dict[augment],
                      "drop", str(dropout), "norm", batch_norm_dict[batch_norm]])
    
    return name;

### Function to train the dataset

In [6]:
def train_wandb(config= None):

    '''best_config = {
        "n_filters": 64,
        "filter_multiplier": 2,
        "augment_data": True,
        "dropout": 0.3,
        "batch_norm": True,
        "epochs": 20,
        "dense_size": 32,
        "lr": 0.001
    }'''
    
    wandb.init(project="Convolutional Neural Networks", entity="cs21s048-cs21s058")
    config = wandb.config
    print(config.augment_data)
    wandb.run.name = setWandbName(config.n_filters, config.filter_multiplier, config.augment_data, config.dropout, config.batch_norm)

    train, val, test = train_dataset(augmentation=config.augment_data, batch_size=config.batch_size)

    model = CNN(n_filters=config.n_filters, filter_multiplier=config.filter_multiplier,
                      dropout= config.dropout, batch_norm = config.batch_norm, dense_size= config.dense_size)
    model.compile(optimizer=keras.optimizers.Adam(config.lr), loss="categorical_crossentropy", metrics="categorical_accuracy")
    model.fit(train, epochs=config.epochs, validation_data=val, callbacks=[WandbCallback()])

In [14]:
def train(config= None):

    train, val, test = train_dataset(augmentation=True, batch_size=16)

    model = CNN(n_filters=256, filter_multiplier=0.5,
                      dropout= 0.3, batch_norm = True, dense_size= 128,act_func= "relu")
    model.compile(optimizer=keras.optimizers.Adam(0.0001), loss="categorical_crossentropy", metrics="categorical_accuracy")
    model.fit(train, epochs=30, validation_data=val)
    print("Testing Model: ")
    model.evaluate(test, batch_size=64)
    
    model.save("best_model.h5")
    model = keras.models.load_model("best_model.h5")

### Predict Function

In [15]:
def predict(model, test_data):
    
    predictions = model(test[0][0])
    model.evaluate(test_data, batch_size=64)
    #plot_images_predicted(test_data, predictions, test_data[0][1])
    return predictions
    #sample_num = 11
    #plot_filters(model, test_generator, sample_num)

### Setting up wandb sweep

In [ ]:
sweep_config = {
    "name": "Final Sweep(Bayesian)",
    "description": "Tuning hyperparameters",
    'metric': {
      'name': 'val_categorical_accuracy',
      'goal': 'maximize'
  },
    "method": "bayes",
    "project": "CS6910_Assignment2",
    "parameters": {
        "n_filters": {
        "values": [16, 32, 64]
        },
        "filter_multiplier": {
            "values": [0.5, 1, 2]
        },
        "augment_data": {
            "values": [True]
        },
        "dropout": {
            "values": [0.3, 0.5]
        },
        "batch_norm": {
            "values": [False, True]
        },
        "epochs": {
            "values": [5, 10]
        },
        "dense_size": {
            "values": [32, 64, 128]
        },
        "lr": {
            "values": [0.01, 0.001]
        },
        "batch_size": {
            "values": [64, 128, 256]
        },
        "activation": {
            "values": ["relu", "leaky"]
        },
    }
}

# creating the sweep
#sweep_id = wandb.sweep(sweep_config, project="Convolutional Neural Networks", entity="cs21s048-cs21s058")

In [ ]:
#wandb.agent("5qv1kqvj", function=train_wandb, count=50)

In [16]:
train()

In [17]:
model = keras.models.load_model("best_model.h5")

In [20]:
predictions = predict(model,test)

### Plotting test data images with predicted labels

In [39]:
idx_to_class = {0: 'Amphibia', 1: 'Animalia', 2: 'Arachnida', 3: 'Aves', 4: 'Fungi', 
                  5: 'Insecta', 6: 'Mammalia', 7: 'Mollusca', 8: 'Plantae', 9: 'Reptilia'}


labels= test[0][1]
plt.figure(figsize=(15,15))
for i in range(30):
        img = test[0][0][i]
        plt.subplot(10,3,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.tight_layout()
        plt.imshow(img)
        #plt.subplots_adjust(bottom=0.2, 
        #            top=0.9, 
        #            wspace=0.4, 
        #            hspace=0.4)
        plt.xlabel("Predicted: " + idx_to_class[np.argmax(predictions, axis=1)[i]] + "\nLabel: " + idx_to_class[np.argmax(labels, axis=1)[i]])
plt.savefig('q4b.png')
plt.show()

In [29]:
#wandb.init(project="Convolutional Neural Networks", entity="cs21s048-cs21s058")

In [31]:
#wandb.log({"example": wandb.Image("q4b.png")})

### Visualize filters of first layer

In [40]:
#Print sizes of all convolution layers:

for layer in model.layers:
    if 'conv' in layer.name:
        filters , bias = layer.get_weights()
        print(layer.name , filters.shape)

### Visualising filters from 1st layer

In [50]:
# get weights from the second hidden layer
filters , bias = model.layers[0].get_weights()

# normalize filter values to 0-1 so we can visualizing
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

n_filters =32
ix=1
fig = plt.figure(figsize=(20,20))

fig, ax = plt.subplots(8, 4, figsize=(15,20))
for i in range(32):
        ax[int(i/4), i%4].imshow(filters[:, :, :, i])
        ax[int(i/4), i%4].axis('off')
        
plt.savefig('q4c.png')

### Visualising feature maps

In [52]:
feature_map_model = Model(inputs=model.inputs, outputs=model.layers[0].output)

#taking an image from test data
plt.imshow(test[0][0][7])
plt.savefig('q4c_2_img.png')
plt.axis('off')

feature_maps = feature_map_model(test[0][0])
fig, ax = plt.subplots(4, 8, figsize=(12,6))
for i in range(32):
    ax[int(i/8), i%8].imshow(feature_maps[7, :, :, i], cmap='gray')
    ax[int(i/8), i%8].axis('off')

plt.savefig('q4c_2.png')

In [48]:
plt.savefig('q4c_2.png')

### Guided Backpropagation

In [60]:
for p in range(10):
    index = p
    file_name = "img_"+str(p)+".jpg"
    # Load the image and pre-process
    image = test[0][0][index]
    image_tensor = np.expand_dims(image, axis=0)

    # Display the image tensor
    plt.imshow(image_tensor[0])
    plt.title("True Label: " + idx_to_class[np.argmax(test[0][1][index])])
    plt.axis("off")
    plt.show()

    #print(image_tensor.shape)

    # we are interested in visulaising the output of 10 neurons in the CONV5 layer
    activation_model = tf.keras.models.Model([model.inputs],[model.get_layer("conv2d_4").output])

    # Creating custom gradients
    @tf.custom_gradient
    def guidedRelU(x):
      def grad(dy):
        return tf.cast(dy>0,"float32") * tf.cast(x>0, "float32") * dy
      return tf.nn.relu(x), grad

    # Apply guided ReLu on activation layers 
    for layer in model.layers[1:]:
        if hasattr(layer, 'activation') and layer.activation == tf.keras.activations.selu:
            layer.activation = guidedRelU

    # Finding gradients of the target class score wrt feature maps
    with tf.GradientTape() as g:
      inputs = tf.cast(image_tensor, tf.float32)
      g.watch(inputs)
      outputs = activation_model(inputs)[0]
    target_gradient = g.gradient(outputs,inputs)[0]

    #print(np.shape(outputs))

    CONV5_layer_activation = outputs

    layer_names = ['conv2d_4']

    #np.shape(CONV5_layer_activation)= (1,size,size,num_features)
    #num_features = CONV5_layer_activation.shape[-1]
    #size = CONV5_layer_activation.shape[1] 
    #print(num_features)
    #print(size)

    num_features = 10
    size = 1
    num_imag_in_row = 1
    num_cols = num_features // num_imag_in_row
    display_grid = np.ones((size * num_cols, num_imag_in_row * size))

    k = 1
    j = 1
    for col in range(num_cols): 
      for row in range(num_imag_in_row):
        channel_image = CONV5_layer_activation[j, k, col * num_imag_in_row + row]
        display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image

    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
    plt.title(layer_names)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto',cmap='gray')  
    plt.show() 

    # Guided backprop gradient visualisation

    grad_image = np.dstack((
                target_gradient[:, :, 0],
                target_gradient[:, :, 1],
                target_gradient[:, :, 2],
            ))       
    grad_image = grad_image - np.min(grad_image)
    grad_image = grad_image/grad_image.max()
    imgplot = plt.imshow(grad_image)
    plt.axis("off")
    plt.savefig(file_name)
    plt.show()

In [55]:
#guided_backprop('best_model.h5', 10, test)